In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

from pytorch_lightning import utilities as pl_utils
from pytorch_lightning.trainer.trainer import Trainer
from pytorch_lightning.plugins import DDPPlugin
import torch
import torch.nn as nn
import pathlib, copy
from functools import partial
import sys, glob
import numpy as np
from timm.models.fx_features import GraphExtractNet
from collections import OrderedDict

sys.path.append('..')
sys.path.append('../deep-learning-base')
sys.path.append('../partially_inverted_reps')

import plot_helper as plt_hp
import output as out
from training import LitProgressBar, NicerModelCheckpointing
import training.finetuning as ft
import architectures as arch
from architectures.callbacks import LightningWrapper, LinearEvalWrapper
from architectures.utils import intermediate_layer_names, FlattenNormalizeConcatenate
from attack.callbacks import AdvCallback
from datasets.data_modules import DATA_MODULES
import datasets.dataset_metadata as dsmd
from partial_loss import PartialInversionLoss, PartialInversionRegularizedLoss
from __init__ import DATA_PATH_IMAGENET, DATA_PATH, SERVER_PROJECT_PATH

/NS/twitter_archive2/work/vnanda/minconda3/envs/dl_base/lib/python3.7/site-packages/paramiko/transport.py:236: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
CHECKPOINT_PATHS = {
    'resnet50': {
        'nonrob': '',
        'robustl2eps3': '/NS/robustness_3/work/vnanda/adv-robustness/logs/'\
                        'robust_imagenet/eps3/resnet-50-l2-eps3.ckpt'},
    'vit_small_patch16_224': {
        'nonrob': \
            '/NS/robustness_2/work/vnanda/invariances_in_reps/deep-learning-base/checkpoints/imagenet21k/'\
            'S_16-i21k-300ep-lr_0.001-aug_light1-wd_0.03-do_0.0-sd_0.0.npz'
        },
    'vit_small_patch32_224': {
        'nonrob': \
            '/NS/robustness_2/work/vnanda/invariances_in_reps/deep-learning-base/checkpoints/imagenet21k/'\
            'S_32-i21k-300ep-lr_0.001-aug_light1-wd_0.03-do_0.0-sd_0.0.npz'
        }
}

In [3]:
SEED = 2
NUM_NODES = 1
DEVICES = 1
BASE_DIR = f"{pathlib.Path('.').parent.resolve()}/checkpoints"

FINETUNING_DATASETS = ['cifar10', 'cifar100', 'flowers', 'oxford-iiit-pets']
FINETUNE_MODE = 'random'
FINETUNE_BS = 256
EVAL_BATCH_SIZE = 100

In [4]:
def calc_acc(dm, m1):
    pl_utils.seed.seed_everything(SEED, workers=True)

    trainer = Trainer(accelerator='gpu', 
                      devices=DEVICES,
                      num_nodes=NUM_NODES,
                      log_every_n_steps=1,
                      auto_select_gpus=True, 
                      deterministic=True,
                      check_val_every_n_epoch=1,
                      num_sanity_val_steps=0,
                      callbacks=[
                        LitProgressBar(['loss', 
                                        'running_test_acc'])])
    out = trainer.predict(m1, dataloaders=[dm.test_dataloader()])
    gt = out[1]
    pred = torch.argmax(out[0], 1)
    return torch.sum(gt == pred) / len(gt)


def get_test_acc(model, source_dataset, append, finetuning_dataset, checkpoint_path, seed, fraction):
    dm = DATA_MODULES[finetuning_dataset](
        data_dir=DATA_PATH_IMAGENET if 'imagenet' in finetuning_dataset else DATA_PATH,
        transform_train=dsmd.TRAIN_TRANSFORMS_TRANSFER_DEFAULT(224),
        transform_test=dsmd.TEST_TRANSFORMS_DEFAULT(224),
        batch_size=EVAL_BATCH_SIZE)
    dm.init_remaining_attrs(source_dataset)

    ## assign mean and std from source dataset
    m_full = arch.create_model(model, source_dataset, pretrained=True,
                               checkpoint_path=checkpoint_path, seed=SEED, 
                               num_classes=dsmd.DATASET_PARAMS[source_dataset]['num_classes'],
                               callback=partial(LightningWrapper, 
                                                dataset_name=source_dataset))
    m_last = arch.create_model(model, source_dataset, pretrained=True,
                               checkpoint_path=checkpoint_path, seed=SEED, 
                               num_classes=dsmd.DATASET_PARAMS[source_dataset]['num_classes'],
                               callback=partial(LightningWrapper, 
                                                dataset_name=source_dataset))
    
    model_full = glob.glob(f'{BASE_DIR}/{model}-base-{source_dataset}-ft-{finetuning_dataset}/'
                                           f'frac-{fraction:.5f}-mode-{FINETUNE_MODE}-seed-{seed}-'
                                           f'ftmode-linear-lr-*-bs*-{append}/'
                                           '*-full-feature-topk=1.ckpt')
    
    model_last = [x for x in glob.glob(f'{BASE_DIR}/{model}-base-{source_dataset}-ft-{finetuning_dataset}/'
                                       f'frac-{fraction:.5f}-mode-{FINETUNE_MODE}-seed-{seed}-'
                                       f'ftmode-linear-lr-*-bs*-{append}/'
                                        '*-topk=1.ckpt') \
                  if 'full-feature' not in x and \
                     'layer' not in x and \
                     'pool' not in x]
    if len(model_full) == 0:
        model_full_acc = np.nan
    else:
        sd_full = torch.load(model_full[0])
        layer_names = intermediate_layer_names(m_full.model)
        chopped_model = GraphExtractNet(copy.deepcopy(m_full.model), layer_names)
        m_full.model = nn.Sequential(
            OrderedDict([
                ('feature_extractor', chopped_model), 
                ('flatten_normalize_concat', FlattenNormalizeConcatenate())
                ]))
        new_layer = ft.setup_model_for_finetuning(
            m_full.model,
            dsmd.DATASET_PARAMS[finetuning_dataset]['num_classes'], FINETUNE_MODE, 
            fraction if fraction != 0. else 0.000001, # edge case where precision wasn't enough in saving filename
            seed, inplace=True, infer_features=True)
        linear_layer = list(m_full.model.named_modules())[-1][1]
        linear_layer.load_state_dict({k.split('.')[-1]:v \
                                      for k,v in sd_full['state_dict'].items()}, strict=True)
        if hasattr(new_layer, 'neuron_indices') and 'neuron_indices' in sd_full:
            assert torch.all(new_layer.neuron_indices == sd_full['neuron_indices'])
        model_full_acc = calc_acc(dm, m_full)

    if len(model_last) == 0:
        model_last_acc = np.nan
    else:
        model_last = model_last[0]
        sd_last = torch.load(model_last)    
        new_layer = ft.setup_model_for_finetuning(
            m_last.model, 
            dsmd.DATASET_PARAMS[finetuning_dataset]['num_classes'],
            FINETUNE_MODE, fraction, seed)
        linear_layer = list(m_last.model.named_modules())[-1][1]
        linear_layer.load_state_dict({k.split('.')[-1]:v \
                                      for k,v in sd_last['state_dict'].items()}, strict=True)
        if hasattr(new_layer, 'neuron_indices') and 'neuron_indices' in sd_last:
            assert torch.all(new_layer.neuron_indices == sd_last['neuron_indices'])
        
        model_last_acc = calc_acc(dm, m_last)
    return model_last_acc, model_full_acc

In [7]:
def results_same_fraction(model, source_dataset):
    plt_str = '== Finetuning results ==\n\n'
    for append in CHECKPOINT_PATHS[model].keys():
        plt_str += f'=== {model} - {append} ===\n\n'
        for ft_ds in FINETUNING_DATASETS:
            plt_str += f'==== {ft_ds} ====\n\n'
            frac_wise_test_accs_last, frac_wise_test_accs_full = {}, {}
            PARTIAL_FRACTIONS = sorted(
                list(set(
                    [float(x.split('/frac-')[1].split('-')[0]) for x in \
                        glob.glob(f'./checkpoints/{model}-base-'
                                  f'{source_dataset}-ft-{ft_ds}/'
                                  f'*-bs-256-{append}') \
                    if len(x.split('/frac-')[1].split('-')[0]) == 7 and \
                    'full-feature' not in x and \
                    'layer' not in x and \
                    'pool' not in x
                    ])))
            for frac in PARTIAL_FRACTIONS:
                PARTIAL_CHOICE_SEEDS = sorted(
                list(set(
                    [int(x.split('-seed-')[1].split('-')[0]) for x in \
                        glob.glob(f'./checkpoints/{model}-base-'
                                  f'{source_dataset}-ft-{ft_ds}/'
                                  f'frac-{frac:.5f}-*-bs-256-{append}')]
                    )))
                for seed in PARTIAL_CHOICE_SEEDS:
                    last_acc, full_acc = get_test_acc(model, source_dataset, append, ft_ds, 
                                CHECKPOINT_PATHS[model][append], seed, frac)
                    print (f'{model} {append} {frac} {ft_ds}, last layer: {last_acc}, full: {full_acc}')
                    if frac in frac_wise_test_accs_last:
                        frac_wise_test_accs_last[frac].append(last_acc)
                        frac_wise_test_accs_full[frac].append(full_acc)
                    else:
                        frac_wise_test_accs_last[frac] = [last_acc]
                        frac_wise_test_accs_full[frac] = [full_acc]

            x_last, y_last = list(zip(*sorted(frac_wise_test_accs_last.items(), key=lambda t: t[0])))
            _, y_full = list(zip(*sorted(frac_wise_test_accs_full.items(), key=lambda t: t[0])))
            plt_str += '{}\n\n'.format(plt_hp.get_wiki_link(plt_hp.line_plot(
                [[np.nanmean(_y) for _y in y_last], [np.nanmean(_y) for _y in y_full]], 
                'Fraction of neurons', 'Transfer Accuracy', ft_ds, 
                subfolder=source_dataset, filename=f'{model}_{ft_ds}_bs_{FINETUNE_BS}_{append}', 
                extension='png', x_vals=x_last, 
                legend_vals=['Last Layer', 'Using Full Features'], 
                vertical_line=None, horizontal_lines=None, 
                horizontal_lines_err=None, colors=plt_hp.COLORS, 
                linestyles=['-', ':'], y_lims=(0.,1.), root_dir='.', 
                paper_friendly_plots=False, plot_inside=False, legend_location='best', 
                savefig=True, figsize=(10,6), marker=[True, True], 
                results_subfolder_name='full_feature_analysis', grid_spacing=None, 
                y_err=[[np.nanstd(_y) for _y in y_last], [np.nanstd(_y) for _y in y_full]], 
                legend_ncol=None), SERVER_PROJECT_PATH, size=700))
    with open(f'./results/full_feature_analysis/{source_dataset}/wiki_results-{model}.txt', 'w') as fp:
        fp.write(plt_str)
    out.upload_results(['{}/{}/{}'.format(plt_hp.RESULTS_FOLDER_NAME, 'full_feature_analysis', source_dataset)], 
        'results', SERVER_PROJECT_PATH, '.png')

In [ ]:
SOURCE_DATASET = 'imagenet'
MODEL = 'resnet50'
results_same_fraction(MODEL, SOURCE_DATASET)

Global seed set to 2
Global seed set to 2
Global seed set to 2
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
Global seed set to 0
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [1]


Predicting: 0it [00:00, ?it/s]

Global seed set to 2


resnet50 nonrob 0.0 cifar10, last layer: nan, full: 0.149399995803833


Global seed set to 2
Global seed set to 2
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
Global seed set to 0
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [1]


...

Predicting: 0it [00:00, ?it/s]

Global seed set to 2


resnet50 nonrob 0.005 cifar10, last layer: 0.27950000762939453, full: 0.8144000172615051


Global seed set to 2
Global seed set to 2
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
Global seed set to 0
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [1]


Predicting: 0it [00:00, ?it/s]